<a href="https://colab.research.google.com/github/chloevan/gcp_tutorial/blob/master/02_basics/Ch20_logical_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
title: "Ch20 Logical Operations"
date: 2020-05-27T07:30:30+09:00
tags:
  - "구글 클라우드"
  - "Google Cloud"
  - "빅쿼리"
  - "BigQuery"
  - "빅쿼리 파이썬"
  - "빅쿼리 기본"
  - "Mathematical Functions"
  - "BigQuery with Python"
categories:
  - "구글 클라우드"
  - "Google Cloud"
  - "빅쿼리"
  - "BigQuery"
  - "Python"
menu: 
  gcp:
    name: Ch20 Logical Operations
---

## I. 구글 클라우드 설정

본격적인 빅쿼리 실습에 앞서서, Python과 연동하는 예제를 준비하였다. 빅쿼리 시작에 앞서서 선행적으로 클라우드 사용을 해야 한다. 

1. 만약 GCP 프로젝트가 없다면, 계정을 연동한다. Go to [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager)
2. 그리고, 비용결제를 위한 카드를 등록한다. [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) 
3. 마지막으로 BigQuery API를 사용해야 하기 때문에 빅쿼리 API 사용허가를 내준다.[Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery)

위 API를 이용하지 않으면 `Python` 또는 `R`과 연동해서 사용할 수는 없다. 자주 쓰는것이 아니라면 비용은 거의 발생하지 않으니 염려하지 않아도 된다. 비용관리에 대한 자세한 내용은 [BigQuery 권장사항: 비용 관리](https://cloud.google.com/bigquery/docs/best-practices-costs?hl=ko)에서 확인하기를 바란다.  


## II. 사용자 계정 인증
구글 코랩을 사용해서 인증 절차를 밟도록 한다. 아래 소스코드는 변경시키지 않는다. 아래 절차대로 진행하면 된다. `Gmail` 인증 절차와 비슷하다.

In [4]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## III. Logical Operations
- `WHERE` 절의 필터링에 관한 섹션에서 `WHERE` 절은 `AND`, `OR` 및 `NOT`를 포함하는 `BOOL`식 뿐만 아니라 실행 순서를 제어하는 괄호를 포함할 수 있다고 호출한다. 

In [5]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd

project_id = 'bigquerytutorial-274406'
client = bigquery.Client(project=project_id)

temp = client.query('''
  SELECT 
    gender, tripduration
  FROM `bigquery-public-data.new_york_citibike.citibike_trips`
  WHERE (tripduration < 600 AND gender = 'female')
  ''').to_dataframe()

print(temp)

         gender  tripduration
0        female           238
1        female           389
2        female           397
3        female           385
4        female           499
...         ...           ...
5020698  female           412
5020699  female           468
5020700  female           561
5020701  female           450
5020702  female           583

[5020703 rows x 2 columns]


- 다음과 같이 `BOOL` 변수와 비교 연산자를 사용할 수 있다.

In [7]:
temp = client.query('''
  WITH example AS (
    SELECT NULL AS is_vowel, NULL as letter, -1 as position
    UNION ALL SELECT true, 'a', 1
    UNION ALL SELECT false, 'b', 2
    UNION ALL SELECT false, 'c', 3
  )
  SELECT * 
  FROM example 
  WHERE is_vowel != false
  ''').to_dataframe()

print(temp)

   is_vowel letter  position
0      True      a         1


- 그런데, 이번의 예에서와 같이 내장 상수와 비교할 때 `IS` 연산자를 사용하는 것이 더 간단한 경우가 많다.

In [8]:
temp = client.query('''
  WITH example AS (
    SELECT NULL AS is_vowel, NULL as letter, -1 as position
    UNION ALL SELECT true, 'a', 1
    UNION ALL SELECT false, 'b', 2
    UNION ALL SELECT false, 'c', 3
  )
  SELECT * 
  FROM example 
  WHERE is_vowel IS NOT false
''').to_dataframe()

print(temp)

  is_vowel letter  position
0     None   None        -1
1     True      a         1


- 의미상으로는 똑같아 보이겠지만, 결과는 다르다. 
- 비교연산자(`=, !=, <, etc`)의 결과는 `NULL`과의 비교를 위해 `NULL`을 반환하지만, `IS` 연산자는 그렇지 않다. 

## IV. NULL에 대한 추가적인 Tip
- `NULL` 일반적으로 수집되지 않은 결측값 또는 값을 나타낸다. 값이 없고 `0도 아니고`, `빈 문자열`도 아니다.
- 데이터 집합에 `NULL`이 있는 경우 `NULL`과의 비교는 항상 `NULL`을 반환하므로 `WHERE` 절에서 `NULL` 값을 필터링하는 것에 주의 한다. 
- `IS` 연산자를 사용하여 값이 `NULL`인지 확인한다. 
- `BOOL` 변수를 직접 사용하는 것이 더 간단하고 읽기 쉽다. 


In [9]:
temp = client.query('''
  WITH example AS (
    SELECT NULL AS is_vowel, NULL as letter, -1 as position
    UNION ALL SELECT true, 'a', 1
    UNION ALL SELECT false, 'b', 2
    UNION ALL SELECT false, 'c', 3
  )
  SELECT * 
  FROM example 
  WHERE is_vowel
''').to_dataframe()

print(temp)

   is_vowel letter  position
0      True      a         1


 

## V. Reference
Lakshmanan, V. (2018). "Exploring a powerful SQL pattern: ARRAY_AGG, STRUCT and UNNEST" Retrieved from https://www.freecodecamp.org/news/exploring-a-powerful-sql-pattern-array-agg-struct-and-unnest-b7dcc6263e36/.